In [47]:
from dotenv import load_dotenv
import os
from pathlib import Path
from alpaca.data import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd
import pandas_ta as ta
import talib as old_ta
from sqlalchemy import create_engine
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [2]:
local_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(local_dir)
config_file_path = os.path.join(parent_dir,'API.env')
load_dotenv(Path(config_file_path))

True

In [3]:
local_dir = os.path.dirname(os.path.abspath('__file__'))
print(os.path.dirname(local_dir))

/Users/ml1000278614/Documents/PythonProjects/stock_paper_trader


In [4]:
api_key = os.getenv("API_KEY")
api_secret = os.getenv("API_SECRET")
base_url = os.getenv("BASE_URL")

api_key

In [13]:
from google.cloud import storage

In [53]:
data = {'stock':['btc-usd'],'action':['sell'],'price':[2450.19],'date':['01-05-2024']}
df = pd.DataFrame(data)
df

,stock,action,price,date
0,btc-usd,sell,2450.19,01-05-2024


In [54]:
client = storage.Client()

bucket_name = 'trade-result-bucket'
blob_name = 'trade-results.db'

table_name = 'results'
url = 'sqlite:///temp_db_file.db'

# Connect to google cloud bucket
bucket = client.get_bucket('trade-result-bucket')
blob = bucket.blob(blob_name)

# Check if blob exists
if blob.exists():
    blob.download_to_filename("temp_db_file.db")

engine = create_engine(url)

df.to_sql(table_name,con=engine,if_exists='append',index=False)

# Upload updated db file
blob.upload_from_filename("temp_db_file.db")

# Remove File
os.remove("temp_db_file.db")



In [14]:
client = storage.Client()

bucket_name = 'trade-result-bucket'
blob_name = 'trade-results.db'

table_name = 'results'
url = 'sqlite:///temp_db_file.db'

# Connect to google cloud bucket
bucket = client.get_bucket('trade-result-bucket')
blob = bucket.blob(blob_name)

blob.download_to_filename("temp_db_file.db")

engine = create_engine(url)
query = f"SELECT * FROM {table_name}"

pd.read_sql(query,engine)

,stock,action,price,date
0,BTC/USD,None,62489.955,2024-05-08 05:00:00.000000
1,BTC/USD,None,62489.955,2024-05-08 05:00:00.000000


In [5]:
client = CryptoHistoricalDataClient()
stock_client = StockHistoricalDataClient(api_key,api_secret)

In [10]:
request_params = CryptoBarsRequest(
    symbol_or_symbols=["BTC/USD"],
    timeframe=TimeFrame.Day,
    start="2020-01-01"
)

btc_bars = client.get_crypto_bars(request_params=request_params)
df = btc_bars.df

In [13]:
df.reset_index(inplace=True)
df[0:5]

,index,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,0,BTC/USD,2021-01-01 06:00:00+00:00,29255.71,29682.29,28707.56,29676.79,848.874030,29639.0,29316.444625
1,1,BTC/USD,2021-01-02 06:00:00+00:00,29678.34,34200.00,29555.99,33769.52,2144.592516,60152.0,31941.412694
2,2,BTC/USD,2021-01-03 06:00:00+00:00,33769.52,34812.93,32300.61,32908.02,1838.695433,58725.0,33505.269474
3,3,BTC/USD,2021-01-04 06:00:00+00:00,32907.36,33496.03,27900.00,30441.57,2711.189503,69226.0,31267.965121
4,4,BTC/USD,2021-01-05 06:00:00+00:00,30461.84,35851.42,29927.55,35063.00,1756.751333,61880.0,33151.367357


In [86]:
print(ta.macd(df['close'],length=20))

      MACD_12_26_9  MACDh_12_26_9  MACDs_12_26_9
0              NaN            NaN            NaN
1              NaN            NaN            NaN
2              NaN            NaN            NaN
3              NaN            NaN            NaN
4              NaN            NaN            NaN
...            ...            ...            ...
1226   -958.158910      26.629806    -984.788715
1227   -983.568729       0.975989    -984.544718
1228   -869.025747      92.415177    -961.440924
1229   -816.367300     116.058899    -932.426199
1230   -526.896673     324.423621    -851.320294

[1231 rows x 3 columns]


In [6]:
trading_client = TradingClient(api_key,api_secret,paper=True)

In [9]:
market_order_data = MarketOrderRequest(
    symbol='BTCUSD',
    qty=0.23,
    side=OrderSide.BUY,
    time_in_force=TimeInForce.GTC
)
market_order = trading_client.submit_order(
    order_data=market_order_data
)

In [11]:
pos = trading_client.get_open_position('BTCUSD')

market_order_data = MarketOrderRequest(
    symbol='BTCUSD',
    qty=pos.qty,
    side=OrderSide.SELL,
    time_in_force=TimeInForce.GTC
)

market_order = trading_client.submit_order(
    order_data=market_order_data
)